In [1]:
import pandas as pd
import numpy as np
import random
import requests

num_coordinates = 2000

lats = np.random.uniform(low=-90.000, high=90.000, size=num_coordinates)
lngs = np.random.uniform(low=-180.000, high=180.000, size=num_coordinates)
lat_lngs = zip(lats, lngs)
coordinates = list(lat_lngs)
coordinates

[(-36.3006412211195, -62.46293044992996),
 (-23.598711054518844, 70.30453975032702),
 (59.0267334892832, 96.28854910494687),
 (-39.6476240815043, -57.25543775875171),
 (80.45952674769165, -89.38279579470013),
 (84.22069473730818, 96.33503625309032),
 (-15.984390513252478, 76.30516111313455),
 (15.32791932323633, 90.12670892569162),
 (-73.11972215934176, -59.1333677506006),
 (-75.73914874943077, 51.03599791220768),
 (-45.208584840884946, 9.587565246483393),
 (41.10283956878425, -155.95250395167696),
 (10.349782678544628, 1.4435034252776973),
 (70.51342812097562, -121.80227620042976),
 (-70.0814278392907, -139.10356699745148),
 (53.260255627142016, -161.12174029235405),
 (70.14025671378417, 94.43629582771678),
 (-3.0205972045142744, -154.68962912268609),
 (-12.389906229634121, -86.09982851045415),
 (49.280710384651826, -175.36562130002483),
 (49.24559980689489, 125.19511672268533),
 (-4.283757037855068, 30.532995703586636),
 (-14.497020076638734, 111.17706720419687),
 (-58.24885502410262

In [2]:
# import the citipy module.
from citipy import citipy

# Create a list for holding the cities.
cities=[]

#Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count.
len(cities)

745

In [3]:
# Import the time library and the datetime module from the datetime library
import time
from datetime import datetime
from config import weather_api_key

#Create an empty list to hold the weather data.
city_data = []

# Create base url for OpenWeather API
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

#Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in our list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1

    #Create endpoint url for each city.
    city_url = url + "&q=" + city

    #log the url,record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    #Add 1 to the record count.
    record_count += 1

    #Run an API request for each of the cities.
    try:
        #Parse the JSON and retrieve the data.
        city_weather = requests.get(city_url).json()
        #Parse out the needed data.
        city_name = city.title()
        city_country = city_weather['sys']['country']
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_humidity = city_weather['main']['humidity']
        city_cloudiness = city_weather['clouds']['all']
        city_wind_speed = city_weather['wind']['speed']
        city_current_description = city_weather['weather'][0]['description']

        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')

        #Append the city information into city_data list.
        city_data.append({"City": city_name,
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_cloudiness,
                          "Wind Speed": city_wind_speed,
                          "Current Description": city_current_description
                          })
    #If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass
#Indicate the data loading is complete.
print("------------------------------")
print("Data Retrieval Complete       ")
print("------------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | general pico
Processing Record 2 of Set 1 | grand river south east
City not found. Skipping...
Processing Record 3 of Set 1 | boguchany
Processing Record 4 of Set 1 | mar del plata
Processing Record 5 of Set 1 | qaanaaq
Processing Record 6 of Set 1 | khatanga
Processing Record 7 of Set 1 | hithadhoo
Processing Record 8 of Set 1 | port blair
Processing Record 9 of Set 1 | ushuaia
Processing Record 10 of Set 1 | port alfred
Processing Record 11 of Set 1 | cape town
Processing Record 12 of Set 1 | kodiak
Processing Record 13 of Set 1 | natitingou
Processing Record 14 of Set 1 | norman wells
Processing Record 15 of Set 1 | rikitea
Processing Record 16 of Set 1 | bethel
Processing Record 17 of Set 1 | talnakh
Processing Record 18 of Set 1 | faanui
Processing Record 19 of Set 1 | chicama
Processing Record 20 of Set 1 | provideniya
Processing Record 21 of Set 1 | nenjiang
Processing Record 22 of Set 1 |

In [4]:
len(city_data)

698

In [5]:
# Add the data to a list in a dict then at the DF
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,General Pico,AR,2022-03-07 16:14:16,-35.6566,-63.7568,75.67,32,0,18.48,clear sky
1,Boguchany,RU,2022-03-07 16:14:17,58.3814,97.4531,32.29,91,100,9.51,overcast clouds
2,Mar Del Plata,AR,2022-03-07 16:14:17,-38.0023,-57.5575,71.62,78,40,15.99,light rain
3,Qaanaaq,GL,2022-03-07 16:14:18,77.4840,-69.3632,0.68,67,51,7.40,broken clouds
4,Khatanga,RU,2022-03-07 16:14:18,71.9667,102.5000,-30.10,100,65,4.92,broken clouds
5,Hithadhoo,MV,2022-03-07 16:14:19,-0.6000,73.0833,83.41,65,100,6.73,overcast clouds
6,Port Blair,IN,2022-03-07 16:14:19,11.6667,92.7500,80.91,67,43,6.62,scattered clouds
7,Ushuaia,AR,2022-03-07 16:01:34,-54.8000,-68.3000,49.57,55,40,3.38,scattered clouds
8,Port Alfred,ZA,2022-03-07 16:05:07,-33.5906,26.8910,73.90,90,100,15.35,overcast clouds
9,Cape Town,ZA,2022-03-07 16:03:24,-33.9258,18.4232,74.12,69,30,20.38,scattered clouds


In [6]:
# Write DataFrame to CSV
csv_weather_file = "WeatherPy_Database.csv"
# Export CSV file
city_data_df.to_csv(csv_weather_file, index_label="City_ID")